In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="google-t5/t5-small")

KeyboardInterrupt: 

In [ ]:
%pip install flask transformers torch


In [ ]:
from flask import Flask, request, jsonify
from transformers import pipeline
import torch
import threading

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1

# Load the translation model (Ensure this model supports translation)
translator = pipeline("translation", model="google-t5/t5-small", device=device)

# Initialize Flask
app = Flask(__name__)

@app.route('/translate', methods=['POST'])
def translate_text():
    try:
        # Get request data
        data = request.get_json()

        # Validate input
        if not data or 'text' not in data:
            return jsonify({'error': 'No text provided'}), 400

        text = data['text']
        if not isinstance(text, str) or text.strip() == "":
            return jsonify({'error': 'Invalid input. Text must be a non-empty string'}), 400

        # Perform translation
        translated_text = translator(text, max_length=512)

        return jsonify({'translation': translated_text[0]['translation_text']})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run Flask app in a separate thread
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

